# PyBot

Este proyecto esta basado en [Chatbot 101](https://apps.worldwritable.com/tutorials/chatbot/)
[github](https://github.com/lizadaly/brobot/blob/master/broize.py)

In [19]:
import re
import spacy
import random
nlp = spacy.load('es')


In [29]:
#usar la coincidencia de palabras clave simple, similar a cómo se modelaron ELIZA y otras IU de conversación iniciales.
#Ejemplo de saludo
POSIBLE_ENTRADA = ("hola", "que tal", "mucho gusto","que pasa?")
POSIBLE_RESP = ["Hola!", "Hey", "Que onda"]

#Oraciones con las que responderemos si no tenemos idea de lo que el usuario acaba de decir
NO_IDEA = [
    "uh como te sientas hoy",
    "Tu puedes!",
    "Trabaja duro",
    "No pain no gain",
    "Que haras despues de entrenar?",
    "deberias de tomar un descanso",
]
#estado de animo
ANIMO = [
    "Como estas el día de hoy?",
    "Que quieres realizar?"
    "Perfecto"
    "En que te puedo ayudar?"
]

#Comentarios hacerca de ejercicio
COMENTARIOS_RUTINAS = [
    "Pierna",
    "Espalda",
    "Biceps"
    "Triceps",
    "Hombro"
]
REPS = "Has {} repeticiones {} veces".format(random.randint(3, 5),random.randint(8, 15))


In [57]:
#Ejemplo SpaCy
oracion = nlp(u'yo necesito hacer ejercicio para mis piernas, ya que estan muy flacidas, que me recomiendas hacer?')
print("PALABRA \t LEMMA \t POS \t\t TAG \t\t DEP")
for token in oracion:
    print(token.text,"\t\t", token.lemma_, "\t", token.pos_, "\t\t",token.tag_,"\t", token.dep_)

print("\nPALABRA \t TAM \t ALPHA \t STOP WORD")
for token in oracion:
    print(token.text,"\t\t", token.shape_,"\t", token.is_alpha,"\t", token.is_stop)

PALABRA 	 LEMMA 	 POS 		 TAG 		 DEP
yo 		 yo 	 PRON 		 PRON__Case=Nom|Number=Sing|Person=1|PronType=Prs 	 nsubj
necesito 		 necesitar 	 VERB 		 VERB__Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin 	 ROOT
hacer 		 hacer 	 VERB 		 VERB__VerbForm=Inf 	 xcomp
ejercicio 		 ejercicio 	 NOUN 		 NOUN__Gender=Masc|Number=Sing 	 obj
para 		 parir 	 ADP 		 ADP__AdpType=Prep 	 case
mis 		 mi 	 DET 		 DET__Number=Plur|Number[psor]=Sing|Person=1|Poss=Yes|PronType=Prs 	 det
piernas 		 pierna 	 NOUN 		 NOUN__Gender=Fem|Number=Plur 	 obl
, 		 , 	 PUNCT 		 PUNCT__PunctType=Comm 	 punct
ya 		 ya 	 ADV 		 ADV___ 	 mark
que 		 que 	 SCONJ 		 SCONJ___ 	 fixed
estan 		 estan 	 VERB 		 VERB__Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin 	 advcl
muy 		 muy 	 ADV 		 ADV___ 	 advmod
flacidas 		 flacidas 	 ADJ 		 ADJ__Gender=Fem|Number=Plur|VerbForm=Part 	 obj
, 		 , 	 PUNCT 		 PUNCT__PunctType=Comm 	 punct
que 		 que 	 SCONJ 		 SCONJ___ 	 nsubj
me 		 me 	 PRON 		 PRON__Number=Sing|Person=1|PronTyp

In [25]:

def saludo(sentencia):
    """si una de las palabras que el usuario introduce coincide regresa una respuesta random"""
    for palabra in sentencia:
        if palabra.text.lower() in POSIBLE_ENTRADA:
            return random.choice(POSIBLE_RESP)

doc = nlp(u'Hola Chatbot!!')

print (saludo(doc))

Que onda


Basicamente si coincide una palbra de la oración te arroja una respuesta. Una primera forma de entablar una conversacion aunque un poco tonta.